# Каким образом будем генерить фичи

In [ ]:
class DataRaif:
    def __init__(self, df):
        self.df = df
        
    def Kolyan1(self):
        t = self.df.iloc[:,0] + self.df.iloc[:,1]
        self.df['feature_name'] = t
    #    return self.df
    
    ## и т.д.
    
    @property
    def get_data(self):
        return self.df

In [ ]:
df = DataRaif(result)

In [ ]:
df.Kolyan1()

In [ ]:
df.get_data

# Normal Pipeline

Pipeline позволяет обернуть сразу несколько этапов обработки данных в один объект

- **FunctionTransformer**: преобразует функцию в трансформер
- **FeatureUnion**: объединяет результаты нескольких трансформеров в один датасет
- **ColumnTransformer**: выбирает определенные колонки

In [2]:
import random
import numpy as np
import os

def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
fix_all_seeds(42)

In [ ]:
numerical = ['']
dummies = ['']
categorical = [col for col in data.columns if col not in numerical+target+dummies]

In [ ]:
class StandardScalerDf(StandardScaler): 
    """
    Позволяет не терять названия колонок при нормализации.
    
    """
    def __init__(self, copy=True, with_mean=True, with_std=True):
        super(StandardScalerDf, self).__init__(copy=copy,
                                               with_mean=with_mean,
                                               with_std=with_std)
        self.feature_names = None

    def transform(self, X, y=None):
        self.feature_names = X.columns
        z = super(StandardScalerDf, self).transform(X.values)
        return pd.DataFrame(z, index=X.index, columns=self.feature_names)
    
    def get_feature_names(self):
        return self.feature_names

In [41]:
scaler_df = StandardScalerDf()

In [ ]:
numerical_selector = FunctionTransformer(lambda df: df[numerical], validate=False)

In [ ]:
numerical_preprocessor = Pipeline([
    ("numerical_selector", numerical_selector),
    ("scaling", scaler_df)
])

In [ ]:
dummy_selector = FunctionTransformer(lambda data: data[dummies], validate=False)

In [ ]:
class OneHotEncodingDf(OneHotEncoder):
    """DataFrame Wrapper around OneHotEncoding"""
    def __init__(self, drop=None, sparse=False, handle_unknown='ignore'):
        super(OneHotEncodingDf, self).__init__(drop=drop,
                                               sparse=sparse,
                                               handle_unknown=handle_unknown
                                              )

    def transform(self, X, y=None):
        z = super(OneHotEncodingDf, self).transform(X.values)
        return pd.DataFrame(z, index=X.index, columns=self.get_feature_names())

In [ ]:
ohe_df = OneHotEncodingDf()

In [ ]:
dummy_preprocessor = Pipeline([
    ("dummy_selector", dummy_selector),
    ("ohe", ohe_df)
])

In [ ]:
categorical_selector = FunctionTransformer(lambda data: data[categorical], validate=False)

In [ ]:
categorical_preprocessor = Pipeline([
    ("categorical_selector", categorical_selector),
    ("ohe", IterativeImputer)
])

# Feature Union

In [ ]:
class PandasFeatureUnion(FeatureUnion):
    def fit_transform(self, X, y=None, **fit_params):
        self._validate_transformers()
        result = Parallel(n_jobs=self.n_jobs)(
            delayed(_fit_transform_one)(
                transformer=trans,
                X=X,
                y=y,
                weight=weight,
                **fit_params)
            for name, trans, weight in self._iter())

        if not result:
            # All transformers are None
            return np.zeros((X.shape[0], 0))
        Xs, transformers = zip(*result)
        self._update_transformer_list(transformers)
        if any(sparse.issparse(f) for f in Xs):
            Xs = sparse.hstack(Xs).tocsr()
        else:
            Xs = self.merge_dataframes_by_column(Xs)
        return Xs

    def merge_dataframes_by_column(self, Xs):
        return pd.concat(Xs, axis="columns", copy=False)

    def transform(self, X):
        Xs = Parallel(n_jobs=self.n_jobs)(
            delayed(_transform_one)(
                transformer=trans,
                X=X,
                y=None,
                weight=weight)
            for name, trans, weight in self._iter())
        if not Xs:
            # All transformers are None
            return np.zeros((X.shape[0], 0))
        if any(sparse.issparse(f) for f in Xs):
            Xs = sparse.hstack(Xs).tocsr()
        else:
            Xs = self.merge_dataframes_by_column(Xs)
        return Xs

In [ ]:
feature_union = PandasFeatureUnion([
    ("numerical_preprocessor", numerical_preprocessor),
    ("dummy_preprocessor", dummy_preprocessor),
    ("categorical_preprocessor", categorical_selector)
])

# Column Transformer (если хочется предопработку фичей закинуть в один пайп)

In [ ]:
cols_preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScalerDf(), numerical),
    ('cat', OneHotEncoder(sparse=False, handle_unknown='ignore'), dummies),
    ('ordinal', 'passthrough', categorical)
], remainder='drop', n_jobs=-1)


In [ ]:
pipeline = Pipeline([
#     ("preprocessing", feature_union),
    ("preprocessing", cols_preprocessor),
    ("modelling", LogisticRegression())
])

In [ ]:
make_column_selector('^((?!Happiness).)*$', dtype_exclude=["object"])(data)

In [ ]:
make_column_selector(dtype_include="float64")(data)

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScalerDf(), make_column_selector(r'^((?!Happiness).)*$', dtype_exclude=["object"])),
    ('cat', OneHotEncoder(sparse=False, handle_unknown='ignore'), make_column_selector(dtype_include="object"))
], remainder='passthrough')

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScalerDf(), numerical),
    ('cat', OneHotEncoder(sparse=False, handle_unknown='ignore'), to_dummies),
    ('ordinal', 'passthrough', categorical)
], remainder='drop', n_jobs=-1)


In [ ]:
# remainer принимает 2 значения: 'drop' и 'passthrough'
# drop дропает такие фичи, либо дропает не заданные в ColumnTransformer
# passthrough ничего с фичами не делает, оставляет в датасете как есть 

# Импутеры разные почекать

In [ ]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp_mean = IterativeImputer(random_state=0)
imp_mean.fit([[7, 2, 3], [4, np.nan, 6], [10, 5, 9]])

X = [[np.nan, 2, 3], [4, np.nan, 6], [10, np.nan, 9]]
imp_mean.transform(X)

In [ ]:
import numpy as np
from sklearn.impute import KNNImputer
X = [[1, 2, np.nan], [3, 4, 3], [np.nan, 6, 5], [8, 8, 7]]
imputer = KNNImputer(n_neighbors=2)
imputer.fit_transform(X)

weights{‘uniform’, ‘distance’}

In [ ]:
sklearn.set_config(display='diagram') #Подключим визуал

# Если нужно задебажить или залезть в какой-то слой

In [ ]:
cols_preprocessor.named_steps['modelling']

In [ ]:
make_column_selector('^((?!per_square_meter_price).)*$', dtype_exclude=["object"])(data)

In [ ]:
make_column_selector(dtype_include="float64")(data)

# Чутка кластеризации

In [ ]:
import os
from sklearn.manifold import TSNE

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing

pd.options.display.max_rows = 20
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import glob
from IPython.display import display, HTML

In [ ]:
from sklearn.manifold import TSNE
from sklearn.metrics import pairwise_distances

distance_matrix_doc = pairwise_distances(train_clustring, train_clustring, metric='cosine', n_jobs=-1)
tsne_doc = TSNE(metric="precomputed", n_components=2, verbose=1, perplexity=30, n_iter=1000, random_state = 42)
tsne_results_doc = tsne_doc.fit_transform(distance_matrix_doc)

In [ ]:
tsne_doc_features = pd.DataFrame({'user_id':train_clustring.index.values})
tsne_doc_features['tsne-2d-one'] = tsne_results_doc[:,0]
tsne_doc_features['tsne-2d-two'] = tsne_results_doc[:,1]

In [ ]:
plt.figure(figsize=(16, 16))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    data=tsne_doc_features,
    legend="full",
    alpha=0.3
)
plt.show()

In [ ]:
# Computing silhouette scores for different clustering option
from tqdm.auto import tqdm
silhouette_list = []
for k in tqdm(range(2, 20, 2)):
    clusters = KMeans(n_clusters=k).fit(train_clustring).labels_.astype(float)
    silhouette_avg = silhouette_score(train_clustring, clusters, metric="euclidean")
    silhouette_list.append(silhouette_avg)
    print(f'Silhouette score for {k} clusters is : {silhouette_avg:.4}')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(2, 20, 2), silhouette_list)

# Сделаем n кластеров

In [ ]:
n = 228
clusters = KMeans(n_clusters=n).fit(train_clustring).labels_.astype(float)

In [ ]:
tsne_doc_features['cluster'] = clusters

In [ ]:
plt.figure(figsize=(16, 16))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    data=tsne_doc_features,
    legend="full",
    alpha=0.3,
    hue='cluster',
    palette='Set1'
)
plt.show()

# Stacking

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

class DjStacking(BaseEstimator, ClassifierMixin):  
    """Стэкинг моделей scikit-learn"""

    def __init__(self, models, ens_model):
        """
        Инициализация
        models - базовые модели для стекинга
        ens_model - мета-модель
        """
        self.models = models
        self.ens_model = ens_model
        self.n = len(models)
        self.valid = None
        
    def fit(self, X, y=None, p=0.25, cv=3, err=0.001, random_state=None):
        """
        Обучение стекинга
        p - в каком отношении делить на обучение / тест
            если p = 0 - используем всё обучение!
        cv  (при p=0) - сколько фолдов использовать
        err (при p=0) - величина случайной добавки к метапризнакам
        random_state - инициализация генератора
            
        """
        if (p > 0): # делим на обучение и тест
            # разбиение на обучение моделей и метамодели
            train, valid, y_train, y_valid = train_test_split(X, y, test_size=p, random_state=random_state)
            
            # заполнение матрицы для обучения метамодели
            self.valid = np.zeros((valid.shape[0], self.n))
            for t, clf in enumerate(self.models):
                clf.fit(train, y_train)
                self.valid[:, t] = clf.predict(valid)
                
            # обучение метамодели
            self.ens_model.fit(self.valid, y_valid)
            
        else: # используем всё обучение
            
            # для регуляризации - берём случайные добавки
            self.valid = err*np.random.randn(X.shape[0], self.n)
            
            for t, clf in enumerate(self.models):
                # это oob-ответы алгоритмов
                self.valid[:, t] += cross_val_predict(clf, X, y, cv=cv, n_jobs=-1, method='predict')
                # но сам алгоритм надо настроить
                clf.fit(X, y)
            
            # обучение метамодели
            self.ens_model.fit(self.valid, y)  
            

        return self
    


    def predict(self, X, y=None):
        """
        Работа стэкинга
        """
        # заполение матрицы для мета-классификатора
        X_meta = np.zeros((X.shape[0], self.n))
        
        for t, clf in enumerate(self.models):
            X_meta[:, t] = clf.predict(X)
        
        a = self.ens_model.predict(X_meta)
        
        return (a)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Базовые алгоритмы для стекинга

### Подбор k для KNN

In [ ]:
error_rate = []

for i in range(1,50):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred = knn.predict(X_test)
    error_rate.append(np.mean(pred != y_test))

plt.figure(figsize=(15,10))
plt.plot(range(1,50),error_rate, marker='o', markersize=9)

# Сделать подбор параметров для алгоритмов через оптюн

In [ ]:
...

In [ ]:
...

In [ ]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

knn1 = KNeighborsRegressor(n_neighbors=3)
knn1.fit(X_train, y_train)
run_and_plot(knn1, X_test, y_test, '3NN')

knn2 = KNeighborsRegressor(n_neighbors=10)
knn2.fit(X_train, y_train)
run_and_plot(knn2, X_test, y_test, '10NN')


rg0 = Ridge(alpha=0.01)
rg0.fit(X_train, y_train)
run_and_plot(rg0, X_test, y_test, 'ridge-0.01')

rg1 = Ridge(alpha=1.1)
rg1.fit(X_train, y_train)
run_and_plot(rg1, X_test, y_test, 'ridge-1.1')

rg2 = Ridge(alpha=100.1)
rg2.fit(X_train, y_train)
run_and_plot(rg2, X_test, y_test, 'ridge-100.1')


rf1 = RandomForestRegressor(n_estimators=100, max_depth=1)
rf1.fit(X_train, y_train)
run_and_plot(rf1, X_test, y_test, 'rf-d1')

rf2 = RandomForestRegressor(n_estimators=100, max_depth=5)
rf2.fit(X_train, y_train)
run_and_plot(rf2, X_test, y_test, 'rf-d5')


gbm1 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=2, n_estimators=200, nthread=-1, objective='regression')    
gbm1.fit(X_train, y_train)
run_and_plot(gbm1, X_test, y_test, 'gbm-d2')

gbm2 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=5, n_estimators=200, nthread=-1, objective='regression')    
gbm2.fit(X_train, y_train)
run_and_plot(gbm2, X_test, y_test, 'gbm-d5')

# Запуск

In [ ]:
models = [knn1, knn2,rg1, rg2, rf1, rf2, gbm1, gbm2] # , rf3
ens_model = Ridge()
s1 = DjStacking(models, ens_model)
s1.fit(train_X, train_y)
run_and_plot(s1, test_X, test_y, '1-stacking')


s2 = DjStacking(models, ens_model)
s2.fit(train_X, train_y, p=-1)
run_and_plot(s1, test_X, test_y, '2-stacking')

# Несколько блендингов подряд

In [ ]:
# models = [knn3, knn5, knn10, rg0, rg1, rg2, rf1, rf2, rf3]
ens_model = Ridge(0.001)
s1 = DjStacking(models, ens_model)
a = 0
e = []
for t in range(10):
    s1.fit(train_X, train_y, p=0.4)
    a += s1.predict(test_X, train_y)
    
    auc = roc_auc_score(test_y, a)
    print (auc)
    e.append(auc)

# Варьируем число фолдов

In [ ]:
# models = [knn3, knn5, knn10, rg0, rg1, rg2, rf1, rf2, rf3]
ens_model = Ridge(0.001)

s1 = DjStacking(models, ens_model)
a = 0
e = []
for t in range(2, 11):
    s1.fit(train_X, train_y, p=-1, cv=t, err=0.00)
    a = s1.predict(test_X, train_y)
    auc = roc_auc_score(test_y, a)
    print (auc)
    e.append(auc)

# Если хочется перебирать параметры прям в пайплайне

In [ ]:
param_grid = {
    'modelling__C': np.logspace(-5, 2, 20),
    'modelling__penalty': ['l1', 'l2'],
    'modelling__class_weight':['balanced', None],
    'preprocessing__num__with_mean':[True, False]
}


search = RandomizedSearchCV(pipeline, param_grid, n_jobs=-1, cv=5)

search.fit(data_train.drop(target, axis=1), data_train[target])
search.best_params_

In [ ]:
f1 = make_scorer(f1_score, average='macro')

param_grid = {
    'modelling__C': np.logspace(-5, 2, 20),
    'modelling__penalty': ['l1', 'l2'],
    'modelling__class_weight':['balanced', None],
    'preprocessing__num__with_mean':[True, False]
}
search = RandomizedSearchCV(pipeline, param_grid, n_jobs=-1, cv=5, scoring=f1)

search.fit(data_train.drop(target, axis=1), data_train[target])
search.best_params_

# Save the pipeline

In [ ]:
with open("pipeline.pkl", "wb") as f:
    pickle.dump(search.best_estimator_, f)

# Load and predict

In [ ]:
with open("pipeline.pkl", 'rb') as f:
    pipeline_from_saved = pickle.load(f)

In [ ]:
pipeline_from_saved

In [ ]:
data_test.iloc[0]

In [ ]:
pipeline_from_saved.named_steps['preprocessing'].transform(data_test.iloc[:1])

In [ ]:
preds = pipeline_from_saved.predict(data_test)
print(classification_report(data_test[target], preds))